In [2]:
import numpy as np
import pandas as pd

from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import stanza

# Load Data and Models

## Tweet Data

In [ ]:
file_name = "exp1.csv"
df = pd.read_csv("exp_datasets/", file_name)

## Sentiment Classifier

In [ ]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

def classify_sentiment(text) :
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    # [ negative, neutral, positive ]
    return scores

## Constituency Parser

In [ ]:
parser = stanza.Pipeline(lang='en', processors='tokenize,pos,constituency')

# Brute-Force Algorithm for Threshold

In [ ]:
def sentiment_distance(tweet, v, np):
  """Calculates the sentiment distance (euclidic distance) between the sentiment scores of the V and NP of a given tweet."""

  tweet_sentiment = classify_sentiment(tweet)
  tweet_no_v_sentiment = classify_sentiment(tweet.replace(v, "")) ##TO DO: change (ask Samba)
  tweet_no_np_sentiment = classify_sentiment(tweet.replace(np, "")) ##TO DO: change (ask Samba)
  v_sentiment = tweet_sentiment - tweet_no_v_sentiment
  np_sentiment = tweet_sentiment - tweet_no_np_sentiment

  return np.linalg.norm(v_sentiment - np_sentiment)

def is_sarc(sentiment_distance, threshold):
  """Returns True if the sentiment distance is greater than a given threshold and False otherwise."""

  return sentiment_distance > threshold

def define_threshold(tweets, gold_annotations, threshold_list):
  """
  Creates a list of tuples containing 1) the tweet body, 2) the sarc/non-sarc gold annotation, 3) the sentiment distance between V and NP.
  Then, iterates over a list of thresholds and for each threshold calculates the accuracy between the gold annotations and the predicted values.
  Returns a dictionary of thresholds and accuracies for those thresholds.
  """

  tweet_annotation_sentiment_distance = []
  for tweet, annotation in zip(tweets, gold_annotations):
    tweet_split = tweet.split()
    v = tweet_split[1]
    np = " ".join(tweet_split[2:])
    tweet_annotation_sentiment_distance.append((tweet, annotation, sentiment_distance(tweet, v, np)))

  # scaling of the data
  sentiment_max = max([x[2] for x in tweet_annotation_sentiment_distance])
  sentiment_min = 0
  tweet_annotation_sentiment_distance = [(t,a, (v - sentiment_min) / (sentiment_max - sentiment_min)) for t,a,v in tweet_annotation_sentiment_distance]

  threshold_accuracy_dict = {}
  for threshold in threshold_list:
    correct = 0
    for _, annotation, sentiment_distance in tweet_annotation_sentiment_distance:
      if is_sarc(sentiment_distance, threshold):
        correct += 1

    threshold_accuracy_dict[threshold] = correct / len(tweet_annotation_sentiment_distance)

  return threshold_accuracy_dict

[('I hate you so much @Eagles', 1, 1.0), ('I love a Monday morning so glad the weekends over!', 1, 0.0), ('i love 6 hour panic attacks', 1, 0.76517206)]
0.0


In [ ]:
gap = 0.001
threshold_list = list(np.arange(0, 1 + gap, gap))

threshold_accuracy_dict = define_threshold(df["tweet"], df["sarcastic"], threshold_list)
print(max(threshold_accuracy_dict, key=threshold_accuracy_dict.get))

In [ ]:
threshold_accuracy_dict

{0.0: 0.6666666666666666,
 0.001: 0.6666666666666666,
 0.002: 0.6666666666666666,
 0.003: 0.6666666666666666,
 0.004: 0.6666666666666666,
 0.005: 0.6666666666666666,
 0.006: 0.6666666666666666,
 0.007: 0.6666666666666666,
 0.008: 0.6666666666666666,
 0.009000000000000001: 0.6666666666666666,
 0.01: 0.6666666666666666,
 0.011: 0.6666666666666666,
 0.012: 0.6666666666666666,
 0.013000000000000001: 0.6666666666666666,
 0.014: 0.6666666666666666,
 0.015: 0.6666666666666666,
 0.016: 0.6666666666666666,
 0.017: 0.6666666666666666,
 0.018000000000000002: 0.6666666666666666,
 0.019: 0.6666666666666666,
 0.02: 0.6666666666666666,
 0.021: 0.6666666666666666,
 0.022: 0.6666666666666666,
 0.023: 0.6666666666666666,
 0.024: 0.6666666666666666,
 0.025: 0.6666666666666666,
 0.026000000000000002: 0.6666666666666666,
 0.027: 0.6666666666666666,
 0.028: 0.6666666666666666,
 0.029: 0.6666666666666666,
 0.03: 0.6666666666666666,
 0.031: 0.6666666666666666,
 0.032: 0.6666666666666666,
 0.033: 0.66666666666